# Graphs

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import joblib

import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3

from sklearn.metrics import confusion_matrix as con_mat

from itertools import chain

### Performance as distance increases

In [ ]:
def plot_dist(df, orientation):
    camera_systems = df['camera system'].unique()
    
    results = df[
        (df['camera system'] == 'single camera') &
        (df['orientation'] == orientation)
    ]

    stereo = df[
        (df['camera system'] == 'stereo') &
        (df['orientation'] == orientation)
    ]

    distances = results['distance'].values
    balanced_accuracy = [float(str(val).split(' ')[0]) for val in results['balanced accuracy'].values]
    roc_auc = [float(str(val).split(' ')[0]) for val in results['roc auc'].values]

    segment_sizes = results['segment size'].values
    segment_overlaps = results['segment overlap'].values

    tele_segment_size = stereo['segment size'].values
    tele_segment_overlap = stereo['segment overlap'].values

    stereo_balanced_accuracy = float(str(stereo['balanced accuracy'].values[0]).split(' ')[0])
    stereo_roc_auc = float(str(stereo['roc auc'].values[0]).split(' ')[0])

    indexes = np.argsort(distances)
    distances = np.array(distances)[indexes]
    balanced_accuracy = np.array(balanced_accuracy)[indexes]
    roc_auc = np.array(roc_auc)[indexes]

    distances = distances / 1000
    return (
        distances, 
        balanced_accuracy, 
        roc_auc, 
        stereo_balanced_accuracy, 
        stereo_roc_auc,
        segment_sizes,
        segment_overlaps,
        tele_segment_size,
        tele_segment_overlap
    )

In [ ]:
orientation = 'Y-Z'

path = 'RESULTS-2D.xlsx'
df = pd.read_excel(path, sheet_name='Sheet1')
(
    distances, 
    balanced_accuracy, 
    roc_auc, 
    stereo_balanced_accuracy, 
    stereo_roc_auc,
    segment_sizes,
    segment_overlaps,
    tele_segment_size,
    tele_segment_overlap
) = plot_dist(df, orientation)

path = 'results-test.xlsx'
df = pd.read_excel(path, sheet_name='Sheet1')
(
    distances_fah, 
    balanced_accuracy_fah, 
    roc_auc_fah, 
    stereo_balanced_accuracy_fah, 
    stereo_roc_auc_fah,
    segment_sizes_fah,
    segment_overlaps_fah,
    tele_segment_size_fah,
    tele_segment_overlap_fah
) = plot_dist(df, orientation)

path = 'results-test-1.xlsx'
df = pd.read_excel(path, sheet_name='Sheet1')
(
    distances_3d, 
    balanced_accuracy_3d, 
    roc_auc_3d, 
    stereo_balanced_accuracy_3d, 
    stereo_roc_auc_3d,
    segment_sizes_3d,
    segment_overlaps_3d,
    tele_segment_size_3d,
    tele_segment_overlap_3d
) = plot_dist(df, orientation)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,10), dpi=600)

ax[0,0].plot(distances, balanced_accuracy, label='2D - Tuned Parameters', color='red')
ax[0,0].plot(distances, [stereo_balanced_accuracy for _ in distances], '--', label='Telecentric - Tuned Parameters', color='orange')
ax[0,0].plot(distances_fah, balanced_accuracy_fah, label='2D - FaH Parameters', color='blue')
ax[0,0].plot(distances_fah, [stereo_balanced_accuracy_fah for _ in distances_fah], '--', label='Telecentric - FaH Parameters', color='green')
ax[0,0].set_xlabel('Distance (m)')
ax[0,0].set_ylabel('Balanced Accuracy score')
ax[0,0].set_title('Balanced Accuracy')
ax[0,0].legend()

ax[0,1].plot(distances, roc_auc, label='2D - Tuned Parameters', color='red')
ax[0,1].plot(distances, [stereo_roc_auc for _ in distances], '--', label='Telecentric - Tuned Parameters', color='orange')
ax[0,1].plot(distances_fah, roc_auc_fah, label='2D - FaH Parameters', color='blue')
ax[0,1].plot(distances_fah, [stereo_roc_auc_fah for _ in distances_fah], '--', label='Telecentric - FaH Parameters', color='green')
ax[0,1].set_xlabel('Distance (m)')
ax[0,1].set_ylabel('ROC AUC score')
ax[0,1].set_title('ROC AUC')
ax[0,1].legend()

ax[1,0].plot(distances, segment_sizes/25, label='2D - Tuned Parameters', color='red')
ax[1,0].plot(distances, [tele_segment_size/25 for _ in distances], '--', label='Telecentric - Tuned Parameters', color='orange')
ax[1,0].plot(distances_fah, segment_sizes_fah/25, label='2D - FaH Params', color='blue')
ax[1,0].plot(distances_fah, [tele_segment_size_fah/25 for _ in distances_fah], '--', label='Telecentric - FaH Parameters')
ax[1,0].set_xlabel('Distance (m)')
ax[1,0].set_ylabel('Segment size (s)')
ax[1,0].set_title('Segment size')
ax[1,0].legend()

ax[1,1].plot(distances, segment_overlaps/25, label='2D - Tuned Parameters', color='red')
ax[1,1].plot(distances, [tele_segment_overlap/25 for _ in distances], '--', label='Telecentric - Tuned Parameters', color='orange')
ax[1,1].plot(distances_fah, segment_overlaps_fah/25, label='2D - Fah Parameters', color='blue')
ax[1,1].plot(distances_fah, [tele_segment_overlap_fah/25 for _ in distances_fah], '--', label='Telecentric - FaH Parameters')
ax[1,1].set_xlabel('Distance (m)')
ax[1,1].set_ylabel('Segment overlap (s)')
ax[1,1].set_title('Segment overlap')
ax[1,1].legend()

plt.show()

### Confusion Matrix

In [ ]:
path = 'E:/2d-anomaly/FILTER/3D/3d-results.dat'
results_3d = joblib.load(path)

path = 'E:/2d-anomaly/FILTER/2m/results.dat'
results_2m = joblib.load(path)

path = 'E:/2d-anomaly/FILTER/15m/results.dat'
results_15m = joblib.load(path)

path = 'E:/2d-anomaly/FILTER/telecentric/results.dat'
results_tele = joblib.load(path)

confusion_data = [results_3d[5], results_tele[5], results_2m[5], results_15m[5]]

In [ ]:
def plot(data, title):
    plt.rcParams.update({'font.size': 12})
    fig, axes = plt.subplots(2, 2, figsize=(10, 8), dpi=600)
    for index, all_test_predictions in enumerate(data):
        y_test =  list(chain.from_iterable(all_test_predictions['true']))
        y_pred =  list(chain.from_iterable(all_test_predictions['preds']))

        con = con_mat(y_test, y_pred)
        con_percentage = con / np.sum(con) * 100
        con_percentage_annot = np.array([["{:.1f}%".format(value) for value in row] for row in con_percentage])
        
        cmap = sns.light_palette("#dd7301", as_cmap=True)
        ax1 = sns.heatmap(con_percentage, annot=con_percentage_annot, cmap=cmap, fmt="", ax=axes[index//2, index%2], cbar=True, vmin=0, vmax=50)
        ax1.set_title(title[index])
        axes[index//2, index%2].set_xlabel("Predicted Class")
        axes[index//2, index%2].set_ylabel("True Class")
        axes[index//2, index%2].set_xticks([0.5,1.5], ['Non-Male', 'Male'])
        axes[index//2, index%2].set_yticks([0.5,1.5], ['Non-Male', 'Male'])
    plt.tight_layout()
    plt.show()

In [ ]:
title = ['(a)','(b)','(c)','(d)']
plot(confusion_data, title)

### ROC Curves

In [ ]:
path = 'E:/2d-anomaly/FILTER/3D/3d-results.dat'
results_3d = joblib.load(path)

path = 'E:/2d-anomaly/FILTER/2m/results.dat'
results_2m = joblib.load(path)

path = 'E:/2d-anomaly/FILTER/15m/results.dat'
results_15m = joblib.load(path)

path = 'E:/2d-anomaly/FILTER/telecentric/results.dat'
results_tele = joblib.load(path)

roc_data = [results_3d[6], results_tele[6], results_2m[6], results_15m[6]]

In [ ]:
def plot(data, title):
    plt.rcParams.update({'font.size': 12})
    fig, axes = plt.subplots(2, 2, figsize=(10, 10), dpi=600)
    for index, roc_curve in enumerate(data):
        lw = 2
        tprs = []
        base_fpr = np.linspace(0, 1, 101)

        for iindex in range(len(roc_curve['tpr'])):
            axes[index//2, index%2].plot(
                roc_curve['fpr'][iindex],
                roc_curve['tpr'][iindex],
                color="blue",
                alpha=0.15,
                lw=lw,
            )
            tpr = np.interp(base_fpr, roc_curve['fpr'][iindex], roc_curve['tpr'][iindex])
            tpr[0] = 0.0
            tprs.append(tpr)

        tprs = np.array(tprs)
        mean_tprs = tprs.mean(axis=0)
        std = tprs.std(axis=0)

        tprs_upper = np.minimum(mean_tprs + std, 1)
        tprs_lower = mean_tprs - std

        axes[index//2, index%2].plot(base_fpr, mean_tprs, 'b')
        axes[index//2, index%2].fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)

        axes[index//2, index%2].plot([0, 1], [0, 1], color="black", lw=lw, linestyle="--")
        axes[index//2, index%2].set_xlabel("False Positive Rate")
        axes[index//2, index%2].set_ylabel("True Positive Rate")
        axes[index//2, index%2].set_xlim((0,1))
        axes[index//2, index%2].set_ylim((0,1))
        axes[index//2, index%2].set_title(title[index])
        #plt.legend(loc="lower right")
    plt.tight_layout()
    plt.show()

In [ ]:
title = ['(a)','(b)','(c)','(d)']
plot(roc_data, title)

### Track examples 

In [ ]:
path = 'H:/Documents/PhD/mosquito-swarms/2d-anomaly-detection/results/3d/'
males_3d = np.load(path + 'tracks_males.npy', allow_pickle=True)

path = 'H:/Documents/PhD/mosquito-swarms/2d-anomaly-detection/results/2d stereo model performance/'
males_2d_stereo = np.load(path + 'tracks_males.npy', allow_pickle=True)

path = 'H:/Documents/PhD/mosquito-swarms/2d-anomaly-detection/results/2d single model performance/'
males_2d_single = np.load(path + 'tracks_males.npy', allow_pickle=True)

males_2d_single_far_far = np.load('tracks_males.npy', allow_pickle=True)

data = [males_3d, males_2d_stereo, males_2d_single, males_2d_single_far_far]

In [ ]:
track_index = 0

def plot(data, title, trial_id=0):
    plt.rcParams.update({'font.size': 10})
    fig, axes = plt.subplots(1, 2, figsize=(13, 6), dpi=600)
    axes[0].axis('off')
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    track = data[0][trial_id][track_index]
    ax.plot(track[:,0], track[:,1], track[:,2])

    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_zlabel('Z (mm)')
    ax.set_title('(a)')

    data = np.array(data)
    for index, trial in enumerate(data[[1,2,3]]):
        index = index +1
        track = trial[trial_id][track_index]
        axes[1].plot(
            (track[:,0] - np.mean(track[:,0])), 
            (track[:,1]- np.mean(track[:,1])), 
            alpha=0.6
        )
        
        axes[1].set_xlabel('Y (mm)')
        axes[1].set_ylabel('Z (mm)')
        axes[1].set_title(title[1])

    axes[1].legend([
        '2D Telecentric', 
        '2D Single Camera at 2m', 
        '2D Single Camera at 15m'
    ])

    plt.tight_layout()
    plt.show()

In [ ]:
plot(data, ['(a)', '(b)'])